In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression.csv', index_col=0)

In [3]:
df['class_id'].mean()

38.63908237747654

In [4]:

# coding: utf-8

# In[1]:

import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger#, TensorBoard
import pickle
from models import AlexNet
from keras import backend as K
K.set_image_data_format('channels_last')


# In[2]:


train_dataset = BatchGenerator(box_output_format=['class_id'])
validation_dataset = BatchGenerator(box_output_format=['class_id'])

train_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_train_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop/',
                        input_format=['image_name', 'class_id'])

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_val_split_47950-70-10-20.csv', 
                             images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop/',
                             input_format=['image_name', 'class_id'])


# In[14]:


img_height, img_width, img_depth = (224,224,3)

epochs = 1

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:


train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())


# In[16]:


steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size




/home/nicoli/github/alexnet/data_generator/batch_generator.py:42: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Number of images in the dataset: 33565
Number of images in the dataset: 4795


In [5]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout

from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [6]:
n_classes=1

In [4]:
if K.image_data_format()=='channels_first':
    input_shape=(3, 224,224)
elif K.image_data_format()=='channels_last':
    input_shape=(224,224,3)

model = Sequential()

model.add(Conv2D(96, (11,11), strides=(4,4), input_shape=input_shape, padding='same', activation='relu', name='conv_1'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

model.add(Conv2D(256, (5,5), strides=(1,1), padding='same', activation='relu', name='conv_2'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='convpool_1'))

model.add(Conv2D(384, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_3'))

model.add(Conv2D(384, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_4'))

model.add(Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_5'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='convpool_5'))

model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='dense_1'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu', name='dense_2'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, name='dense_3'))

In [5]:
from keras.layers.advanced_activations import LeakyReLU

In [6]:
model.add(LeakyReLU(alpha=0.3))

In [7]:
#model.load_weights('callbacks/alexnet/age/weights.63-1679.96.hdf5', by_name=True)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
convpool_1 (MaxPooling2D)    (None, 13, 13, 256)       0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 13, 13, 384)       885120    
__________

In [13]:
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)


# In[17]:
#embedding_layer_names = set(layer.name
#                            for layer in alexnet.model.layers)
#print(embedding_layer_names)

#tensorboard = TensorBoard(log_dir='tensorboard/alexnet/age/', 
#                          histogram_freq=1, #embeddings_freq=1, 
#                          write_images=True)
#                          #embeddings_layer_names=embedding_layer_names)
      
#tensorboard.set_model(alexnet.model)

csv_logger = CSVLogger('callbacks/alexnet/age/history.csv', append=True, separator=',')

checkpoint = ModelCheckpoint(filepath='callbacks/alexnet/age/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=1)
#callbacks = [tensorboard, checkpoint]
callbacks=[checkpoint, csv_logger]


# In[18]:


model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])


# In[19]:


history = model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)


Epoch 1/1
 12/524 [..............................] - ETA: 1:27:14 - loss: 141207163093025136.0000 - mean_absolute_error: 73271638.5100

KeyboardInterrupt: 